# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [8]:
!cat publications.tsv










3/20/21	Correcting for Background Noise Improves Phenotype Prediction from Human Gut Microbiome Data	Biorxiv	We show that application of the centered log-ratio transformation prior to correction with unsupervised approaches improves prediction accuracy for many phenotypes while simultaneously reducing variance due to unwanted sources of variation.	"Briscoe, L., Balliu, B., Sankararaman, S.,Halperin, E., & Garud, N. (2021). Correcting for Background Noise Improves Phenotype Prediction from Human Gut Microbiome Data. bioRxiv 2021.03.19.436199 (2021). doi:10.1101/2021.03.19.436199"	PCA correction	https://doi.org/10.1101/2021.03.19.436199

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [9]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,6/30/16,Symbiotic Burkholderia Species Show Diverse Ar...,International Society for Molecular Plant-Micr...,deeper genomic analysis revealed a complex sym...,"De Meyer, S. E., Briscoe, L., Martinez-Hidalgo...",Diverse arrangements of nitrogen fixation genes,https://doi.org/10.1094/MPMI-05-16-0091-R
1,10/5/17,SaVanT: a web-based tool for the sample-level ...,BMC Genomics,Signature visualization of individual samples ...,"Lopez, D., Montoya, D., Ambrose, M., Lam, L., ...",SaVanT,https://doi.org/10.1186/s12864-017-4167-7
2,7/24/18,Whole genome analyses suggests that Burkholder...,Genes,The newly proposed combinations are Mycetohabi...,"Estrada-de los Santos, P., Palmer, M., Ch vez-...",Novel genera in Burkholderiaceae,https://doi.org/10.3390/genes9080389
3,6/10/19,FEAST: fast expectation-maximization for micro...,Nature Methods,FEAST may provide insight into quantifying con...,"Shenhav, L., Thompson, M., Joseph, T. A., Bris...",FEAST,https://doi.org/10.1038/s41592-019-0431-x
4,6/27/19,Modeling the temporal dynamics of the gut micr...,PLoS Computational Biology,MTV-LMM can identify time-dependent microbes (...,"Shenhav, L., Furman, O., Briscoe, L., Thompson...",MTVLMM,https://doi.org/10.1371/journal.pcbi.1006960
5,10/20/21,Optimized design of single-cell RNA sequencing...,Nature Communications,we provide a practical approach on selecting c...,"Mandric, I., Schwarz, T., Majumdar, A., Hou, K...",cost-effective designs,https://doi.org/10.1038/s41467-020-19365-w
6,3/20/21,Correcting for Background Noise Improves Pheno...,Biorxiv,We show that application of the centered log-r...,"Briscoe, L., Balliu, B., Sankararaman, S.,Halp...",PCA correction,https://doi.org/10.1101/2021.03.19.436199


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [10]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [11]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [12]:
!ls ../_publications/

16-Diverse arrangements of nitrogen fixation genes.md
17-SaVanT.md
18-Novel genera in Burkholderiaceae.md
19-FEAST.md
19-MTVLMM.md
2016-06-30-Diverse arrangements of nitrogen fixation genes.md
2017-10-05-SaVanT.md
2018-07-24-Novel genera in Burkholderiaceae.md
2019-06-10-FEAST.md
2019-06-27-MTVLMM.md
21-PCA correction.md
21-cost-effective designs.md


In [13]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
